## Import Modules

In [ ]:
# Import Modules

import dash
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_core_components as dcc
import dash_html_components as html
import dash_extensions as de
import dash_leaflet as dl
from jupyter_dash import JupyterDash
import dash_leaflet.express as dlx
from dash_extensions.javascript import arrow_function
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Create App

external_stylesheets = [
    'https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css',
    'https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css',
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
]


app = JupyterDash(
    name=__name__,
    external_stylesheets=external_stylesheets,
    prevent_initial_callbacks=True
)


# Create Layout App


keys = {
    "One": {
        "url": "https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}{r}.png",
        "name": "Terrain"
    },
    "Two": {
        "url": "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
        "name": "Open Street Map"
    },
    "Three": {
        "url": "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}",
        "name": "Google Satellite Imagery"
    }
}

AQUIFERS = "./App/static/shapefiles/Aquifers.geojson"
MAHDOUDE = "./App/static/shapefiles/Mahdoude.geojson"
HOZEH6 = "./App/static/shapefiles/Hozeh6.geojson"
HOZEH30 = "./App/static/shapefiles/Hozeh30.geojson"
OSTAN = "./App/static/shapefiles/Ostan.geojson"
SHAHRESTAN = "./App/static/shapefiles/Shahrestan.geojson"


attribution = '&copy; <a href="http://www.khrw.ir/">Khorasan Regional Water Company</a> '



info = html.Div(
    id="info",
    className="info",
    style={
        "position": "absolute",
        "bottom": "10px",
        "left": "10px",
        "zIndex": "1000"
    },
    dir="rtl"
)


search_bar = html.Div(
    children=[
        dcc.Input(
            id="search_lat_lng",
            type="search",
            placeholder="Lat: 36.30 Lon: 59.60",
            debounce=True
        )
    ],
    style={
        "position": "absolute",
        "top": "10px",
        "left": "50px",
        "zIndex": "1000"
    },
    dir="ltr"
)


model = html.Div(
    children=[
        dbc.Modal(
            id="modal",
            centered=True,
            is_open=False,
            children=[
                dbc.ModalHeader(
                    id="modal_header"
                ),
                dbc.ModalBody(
                    id="modal_body"
                )
            ]
        )
    ]
)


app.layout = html.Div([
    
    html.Div(
        children=[
            
            dl.Map(
                id="map",
#                 center=[36.30, 59.60],
                zoom=5,
                children=[
                    dl.LayersControl(
                        children=[
                            dl.BaseLayer(
                                children=[
                                    dl.TileLayer(
                                        url=key["url"],
                                        attribution=attribution
                                    ),
                                ],
                                name=key["name"],
                                checked=key["name"] == "Open Street Map"  
                            ) for key in keys.values()                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.LayerGroup(
                                        id="layer"
                                    )                                    
                                ],
                                name="click+search",
                                checked=True
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="ostan",
                                        url="/assets/Ostan.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "blue"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="استان",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="shahrestan",
                                        url="/assets/Shahrestan.geojson",
#                                         zoomToBounds=True,
                                        zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "#FD8D3C"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="شهرستان",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="hozeh6",
                                        url="/assets/Hozeh6.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "red"
                                            }
                                        },
                                    )                                    
                                ],
                                name="حوضه درجه یک",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="hozeh30",
                                        url="/assets/Hozeh30.geojson",
#                                         zoomToBounds=True,
#                                         zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "green"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="حوضه درجه دو",
                                checked=False
                            )                            
                        ] + [
                            dl.Overlay(
                                children=[
                                    dl.GeoJSON(
                                        id="mahdoude",
                                        url="/assets/Mahdoude.geojson",
                                        zoomToBounds=True,
                                        zoomToBoundsOnClick=True,
                                        hoverStyle=arrow_function(
                                            dict(
                                                weight=5,
                                                color='#222',
                                                dashArray=''
                                            )
                                        ),
                                        options={
                                            "style": {
                                                "color": "tomato"
                                            }
                                        }                                       
                                    )                                    
                                ],
                                name="محدوده مطالعاتی",
                                checked=True
                            )                            
                        ]
                    ),
                    dl.LocateControl(
                        options={
                            'locateOptions': {
                                'enableHighAccuracy': True
                            }
                        }
                    ),
                    dl.MeasureControl(
                        position="topleft",
                        primaryLengthUnit="kilometers",
                        primaryAreaUnit="hectares",
                        activeColor="#214097",
                        completedColor="#972158"
                    ),
                    search_bar,
                    info,
                    model
                ],
                style={
                    'width': '100%', 
                    'height': '75vh',
                    'margin': "auto",
                },
            ),
        ],
        dir="rtl"
    ),
    html.Div(
        children=[
            html.Div(
                id="out1",
                className="col-6"
            ),
            html.Div(
                id="out2",
                className="col-6"
            )
        ],
        className="row"
    )
])


# App Callback

@app.callback(
    Output("layer", "children"),
    Output("search_lat_lng", "value"),
    Output("map", "center"),
    Input("map", "dbl_click_lat_lng"),
    Input("search_lat_lng", "value")
)
def map_dbl_click(dbl_click_lat_lng, search_lat_lng):
    print(search_lat_lng)
   
    if search_lat_lng is None or search_lat_lng == "" and dbl_click_lat_lng:
        result = dl.Marker(
            children=dl.Tooltip(
                "({:.3f}, {:.3f})".format(*dbl_click_lat_lng)
            ),
            position=dbl_click_lat_lng
        )

        return [result], "", dbl_click_lat_lng
    
    else:
        
        search_lat_lng = list(
            filter(
                ("").__ne__,
                search_lat_lng.split(" ")
            )
        )
        
        if len(search_lat_lng) == 2:
        
            search_lat_lng = [float(x) for x in search_lat_lng]

            result = dl.Marker(
                children=dl.Tooltip(
                    "({:.2f}, {:.2f})".format(*search_lat_lng)
                ),
                position=search_lat_lng
            )

            return [result], "", search_lat_lng
        else:
            raise PreventUpdate
    

@app.callback(
    Output("out1", "children"), 
    Input("ostan", "click_feature")
)
def ostan_click(feature):
    if feature is not None:
        return f"شما {feature['properties']['ostn_name']}"
    
@app.callback(
    Output("info", "children"), 
    Input("ostan", "hover_feature"),
    Input("mahdoude", "hover_feature"),
)
def ostan_click(feature_ostan, feature_mahdoude):
    header = [
        html.H4(
            "US Population Density"
        )
    ]
    
    if feature_ostan:
        return header + [
            html.B(feature_ostan["properties"]["ostn_name"]),
            html.Br(),
            "{:.1f} ha".format(feature_ostan["properties"]["AREA"] / 10000)
        ]
    elif feature_mahdoude:
        return header + [
            html.B(f"نام محدوده مطالعاتی: {feature_mahdoude['properties']['MahName']}"),
            html.Br(),
            html.B(f"کد محدوده مطالعاتی: {feature_mahdoude['properties']['MahCode']}"),
            html.Br(),
            html.B(f"حوزه درجه دو: {feature_mahdoude['properties']['Hoze30Name']}"),
                        html.Br(),
            html.B(f"حوزه درجه یک: {feature_mahdoude['properties']['Hoze6Name']}"),
        ]
    else:
        return header + ["Hoover over a state"]


@app.callback(
    Output("ostan", "children"),
    Input("ostan", "hover_feature"),
)
def oss(feature_ostan):
    if feature_ostan is not None:
        return dl.Tooltip(
            f"{feature_ostan['properties']['ostn_name']}"
        )
    else:
        raise PreventUpdate
        

        


df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)        
        

@app.callback(
    Output("modal", "is_open"),
    Output("modal_header", "children"),
    Output("modal_body", "children"),
    Input("ostan", "click_feature"),
    State("modal", "is_open"),
)
def toggle_modal(feature_ostan, is_open):
    if feature_ostan is not None:
        return not is_open, f"{feature_ostan['properties']['ostn_name']}", html.Div([dcc.Graph(id='life-exp-vs-gdp', figure=fig)
])
    else:
        raise PreventUpdate



# Run App

app.run_server(
    mode="external",
)

In [ ]:
import geopandas as gpd
data = gpd.read_file('assets/Hozeh6.geojson')
print(data)

# --------------------------------------------------------------------------------
# Convert Wide Data To Long Data
# --------------------------------------------------------------------------------

In [4]:
import pandas as pd
import numpy as np

WaterTableData = pd.read_csv("WaterTableData.csv")

ThissenAreaData = pd.read_csv("ThissenAreaData.csv")

waterTable = pd.melt(
    WaterTableData,
    id_vars=WaterTableData.columns[0:4].to_list(),
    var_name='Date',
    value_name='wt'
)

waterTable["Date"] = waterTable["Date"].apply(pd.to_datetime)

thissenArea = pd.melt(
    ThissenAreaData,
    id_vars=ThissenAreaData.columns[0:4].to_list(),
    var_name='Date',
    value_name='th'
)

thissenArea["Date"] = thissenArea["Date"].apply(pd.to_datetime)

result = waterTable.merge(
    thissenArea,
    how="outer",
    on=["Mahdodeh_Name", "Mahdodeh_Code", "Aquifer_Name", "Well_Name", "Date"]
).sort_values(by=["Mahdodeh_Name", "Mahdodeh_Code", "Aquifer_Name", "Well_Name", "Date"])

result.columns = ["MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME", "DATE", "WATER_TABLE", "THISSEN_LOCATION"]

# Mean Thissen
tm = result.groupby(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE']).agg({'THISSEN_LOCATION': 'sum'}).reset_index()
tm.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE", "THISSEN_MAHDOUDE"]

result = result.merge(
    tm,
    how="outer",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE"]
).sort_values(by=["MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME", "DATE"])

result['THISSEN_MAHDOUDE'] = result['THISSEN_MAHDOUDE'].replace(0, np.nan)

result.to_excel("HydrographDataRaw.xlsx")

# Calculate Aquifer Hydrograph

In [9]:
import pandas as pd
import numpy as np
import base64
import jalali

# Modify Date And Water Table
# Date Type: 
#   1.Gregorian: 2000-11-1
#   2.Persian: 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(info, date, value, date_type="gregorian"):
    df = info.copy()
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = date.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
    elif date_type == "persian":
        df["DATE_PERSIAN"] = date
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)    
    else:
        pass
    df["VALUE"] = value        
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    df["VALUE_NEW"] = df["VALUE"]
    A = []
    A.append(df["VALUE"][0])
    for i in range(1, len(df)):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    df["VALUE_NEW"] = A
    return df


# Read Data And GeoInfo
xls = pd.ExcelFile('HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')
DATE_TYPE = 'gregorian'


# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())


# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())


# Convert Date
if DATE_TYPE == "gregorian":
    Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)
    Data["DATE_PERSIAN_RAW"] = list(map(lambda i: jalali.Gregorian(i.date()).persian_string(),
                                    Data["DATE_GREGORIAN_RAW"]))
elif DATE_TYPE == "persian":
    Data["DATE_GREGORIAN_RAW"] = list(map(lambda i: jalali.Persian(i).gregorian_string(),
                                        Data["DATE_PERSIAN_RAW"]))
    Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)    
else:
    pass



wt_date_converted = convert_to_day_15(
    info=Data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]],
    date=Data["DATE_PERSIAN_RAW"],
    value=Data["WATER_TABLE_RAW"],
    date_type="persian"
)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW"]
)

Data

,MAHDOUDE_NAME,MAHDOUDE_CODE,AQUIFER_NAME,LOCATION_NAME,DATE_GREGORIAN_RAW,DATE_PERSIAN_RAW,WATER_TABLE_RAW,NO_MEASURE_CODE,INFO,STORAGE_COEFFICIENT,THISSEN_LOCATION,THISSEN_MAHDOUDE,DATE_PERSIAN,DATE_GREGORIAN,WATER_TABLE
0,جوین,4740,جوین,ابراهیم آباد بالا جوین,2001-10-07,1380-7-15,8.027576,NaN,NaN,0.07,27.815186,2607.016926,1380-7-15,2001-10-07,8.027576
1,جوین,4740,جوین,ابراهیم آباد بالا جوین,2001-11-06,1380-8-15,7.728636,NaN,NaN,0.07,27.815186,2607.016926,1380-8-15,2001-11-06,7.728636
2,جوین,4740,جوین,ابراهیم آباد بالا جوین,2001-12-06,1380-9-15,7.900690,NaN,NaN,0.07,27.815186,2607.016926,1380-9-15,2001-12-06,7.900690
3,جوین,4740,جوین,ابراهیم آباد بالا جوین,2002-01-05,1380-10-15,7.901875,NaN,NaN,0.07,27.815186,2607.016926,1380-10-15,2002-01-05,7.901875
4,جوین,4740,جوین,ابراهیم آباد بالا جوین,2002-02-04,1380-11-15,7.970645,NaN,NaN,0.07,27.815186,2607.016926,1380-11-15,2002-02-04,7.970645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25015,مشهد - چناران,6007,مشهد - چناران,یکه لنگه,2016-05-04,1395-2-15,107.497850,NaN,NaN,0.07,29.239296,2422.877519,1395-2-15,2016-05-04,107.497850
25016,مشهد - چناران,6007,مشهد - چناران,یکه لنگه,2016-06-04,1395-3-15,107.580909,NaN,NaN,0.07,29.239296,2422.877519,1395-3-15,2016-06-04,107.580909
25017,مشهد - چناران,6007,مشهد - چناران,یکه لنگه,2016-07-05,1395-4-15,107.635394,NaN,NaN,0.07,29.239296,2422.877519,1395-4-15,2016-07-05,107.635394
25018,مشهد - چناران,6007,مشهد - چناران,یکه لنگه,2016-08-05,1395-5-15,107.847000,NaN,NaN,0.07,29.239296,2422.877519,1395-5-15,2016-08-05,107.847000
